In [31]:
import numpy as np
import math
import operator
import networkx as nx
from itertools import permutations
import matplotlib.pyplot as plt
from deap import creator, base, tools, algorithms,gp
import dimod
import import_ipynb
import Christofides
from Christofides import christofedes,DrawGraph
import dwave_networkx as dnx
from operator import itemgetter
import tsplib95
import copy
import pygraphviz as pgv
from IPython.display import Image 
import sys
from tqdm import tqdm
import pytsp

In [25]:
problem = tsplib95.load('./asymmetric_problems/ft70.atsp')
# problem = tsplib95.load('./symmetric_problems/berlin52.tsp')
G = problem.get_graph()
# tsp_problem = problem.as_keyword_dict()
# list(problem.get_nodes())

In [26]:
pos = nx.random_layout(G)

In [28]:
#dnx.traveling_salesperson(G, dimod.ExactSolver(), start=0)

In [29]:
nx.density(G)

1.0144927536231885

In [ ]:
def plotRandomLayoutGraph(G,show_weights=False):
    pos = nx.random_layout(G)
    nx.draw(G,pos,with_labels=True)
    labels = nx.get_edge_attributes(G,'weight')
    if show_weights:
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)

    plt.show()

In [ ]:
plotRandomLayoutGraph(G)

In [ ]:
def evaluation(individual):
    total_weights = []
    for index in range(len(individual)-1):
        total_weights.append(G.edges[individual[index],individual[index+1]]['weight'])
    return sum(total_weights),

In [ ]:
def create_tour(individual):
    new_graph = nx.Graph()
    new_graphG=nx.complete_graph(len(individual))
    new_graph.clear_edges()
    for index in range(len(individual)-1):
        new_graph.add_edge(individual[index],individual[index+1])
    new_graph.add_edge(individual[len(individual)-1],individual[0])
    
    return new_graph

### GENETIC PROGRAMMING

In [ ]:
def Div(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

In [ ]:
def Mod(left, right):
    try:
        return left % right
    except ZeroDivisionError:
        return 1

In [ ]:
pset = gp.PrimitiveSet("MAIN",5)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(Div, 2)
pset.addPrimitive(Mod, 2)
#Number of nodes in the graph
pset.renameArguments(ARG0="Nn")
#Number of remaining nodes to visit
pset.renameArguments(ARG1="Nrn")
#Distance from the current node
pset.renameArguments(ARG2="Dcn")
#Distance from the initial node
pset.renameArguments(ARG3="Din")
#Predicted distance from the initial node
pset.renameArguments(ARG4="Pd")

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

In [ ]:
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [ ]:
G.edges[0,1]['weight']

In [ ]:
def nearest_neighbor(current,unvisited_nodes, G):
    min_node = min(unvisited_nodes, key=lambda x: G.edges[x, current]["weight"])
    return min_node

In [ ]:
def greedy_TSP(G,visited):
    vis = copy.deepcopy(visited)
    while len(vis) < len(G.nodes):
        C = nearest_neighbor(vis[-1], np.setdiff1d(list(G.nodes),vis),G)
        vis.append(C)
    vis.append(vis[0])
    score = evaluation(vis)
    return score

In [ ]:
greedy_TSP(G,[0])

In [ ]:
def shortest_path(G,P):
    total =0
    for index in range(len(P)-1): 
        total += G.edges[P[index],P[index +1]]["weight"]
    return total

In [ ]:
def pred_shortest_path(G,unvisited,node):
    un_visited = unvisited.tolist()
    un_visited.insert(0,0)
    sub_G = G.subgraph(un_visited)
    return shortest_path(G,nx.dijkstra_path(sub_G,source=node,target=0))
    #return sub_G

In [ ]:
def evaluate(individual,G):    
    tree = gp.PrimitiveTree(individual)
    
    node_results = [0]
    current_node = 0
    initial_node =0
    visited = [0]
    solution = [0]
    unvisited = np.setdiff1d(list(G.nodes),visited)
    while len(unvisited) > 1:
        results = []
        for index,node in enumerate(unvisited):
            if index not in visited:
                Nn = len(G.nodes())
                Nrn = len(np.setdiff1d(list(G.nodes),visited))
                Dcn = G.edges[current_node,index]['weight']
                Din = G.edges[initial_node,index]['weight']
                Pd = pred_shortest_path(G,unvisited,node)
                result = toolbox.compile(expr=individual)
                results.append({"result": result(Nn,Nrn,Dcn,Din,Pd),"node":index})
        if not results:
            break
        nearest = min(results, key=lambda x:x['result'])
        solution.append(nearest["node"])
        current_node = nearest["node"]
        visited.append(nearest["node"])

    solution.append(initial_node)
    return evaluation(solution)

In [ ]:
plotRandomLayoutGraph(G)

In [ ]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=2, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evaluate, G= G)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [ ]:
%%time
evaluate(toolbox.individual(),G)

In [ ]:
fit_stats = tools.Statistics(key=operator.attrgetter("fitness.values"))
fit_stats.register('mean', np.mean)
fit_stats.register('min', np.min)

In [ ]:
pop = toolbox.population(n=50)
pop, log = algorithms.eaSimple(pop, toolbox,
                         cxpb=0.8, mutpb=0.2,
                         ngen=20,stats=fit_stats,verbose=True)

In [ ]:
%%time
min_results =[]
mean_results = []
for index in tqdm(range(10)):
    pop = toolbox.population(n=50)
    pop, log = algorithms.eaSimple(pop, toolbox,
                             cxpb=0.8, mutpb=0.2,
                             ngen=20,stats=fit_stats,verbose=True)
    min_results.append(np.min(log.select('min')))
    mean_results.append(np.min(log.select('mean')))

### BEST INDIVIDUAL

In [ ]:
best_individual = tools.selBest(pop, k=1)[0]
nodes, edges, labels = gp.graph(best_individual)

tree = gp.PrimitiveTree(best_individual)
string_tree = str(tree)
print(tree.height)
# print('Fitness of the best individual: ', evaluation(best_individual)[0])

g = pgv.AGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.layout(prog="dot")

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw("./figures/tree.png")

pil_img = Image(filename='./figures/tree.png')
display(pil_img)

In [ ]:
plt.figure(figsize=(11, 4))
plots = plt.plot(log.select('min'),'c-', log.select('mean'), 'b-')
plt.legend(plots, ('Minimum fitness', 'Mean fitness'), frameon=True)
plt.ylabel('Fitness'); plt.xlabel('Iterations');
plt.title("br17 asymetric graph")

In [ ]:
mean_distance = np.mean(min_results)
min_distance = np.min(min_results)
std_distance = np.std(min_results)

In [ ]:
print("scores,mean:{},min:{},std:{}".format(mean_distance,min_distance,std_distance))